In [1]:
import pandas as pd

deals = pd.read_csv('../../uploads/deals.csv')
organizations = pd.read_csv('../../uploads/organizations.csv')

In [2]:
deals

,Affinity Row ID,Organization Id,Name,Website,Description,Industry,Investors,Sector,Sub-Sector,Tech Trend(s)
0,113980601,117847450,Sheeva.AI (formerly Parkofon),sheeva.ai,"Enjoy the benefits of extreme loyalty, easy pa...",Internet Software & Services; Information Tech...,Techstars,Energy; Transportation + Logistics,Connected Vehicles; EV Ecosystem,Location Tracking
1,113228959,263025514,Mobilyze,mobilyze.ai,Location analytics for the entire EV ecosystem...,Renewable Electricity; Renewables & Environmen...,Plug And Play Tech Center; Urban Us,Transportation + Logistics; Energy,EV Ecosystem,Predictive Analytics and Forecasting
2,112965565,164005770,Tenderd,tenderd.com,Equipment Rental Marketplace for Middle East |...,Construction; Transportation; Logistics; Fleet...,Catapult Vc; Global Founders Capital; Y Combin...,Built Environment,Asset Tracking & Management; Carbon Data Capture,Fintech and Marketplaces
3,112960319,223570042,Advanced Ionics,advanced-ionics.com,Advanced Ionics has created an electrolyzer th...,Gas Utilities; Oil & Gas; Other Energy; Indust...,Southwest Angel Network; Clean Energy Ventures,Energy,Hydrogen,Deeptech
4,112834861,263756234,RailVision Analytics,railvision.ca,"Customized data analysis, software solutions, ...",Internet Software & Services; Information Tech...,Active Impact Investments; Cycle Momentum,Transportation + Logistics,"Marine, Rail, & Port Logistics",Analytics; Artificial Intelligence; Business I...
...,...,...,...,...,...,...,...,...,...,...
86,41128219,131081675,Buzz Solutions,buzzsolutions.co,"Artificial Intelligence, Actionable Insights a...",Internet Software & Services; Software Develop...,Blackhorn Ventures; Advisors.fund; Ulu Venture...,Energy,Transmission & Distribution Maintenance,Analytics
87,40188678,1795902,Telesense,telesense.com,TeleSense is a next-generation grain monitorin...,Transportation; Internet Software & Services; ...,Artesian Investment; Artesian; Mindset Venture...,NaN,NaN,NaN
88,40146103,224928034,Nth Cycle,nthcycle.com,Nth Cycle is a metals process technology compa...,Metals & Mining; Commercial & Residential Cons...,Mercuria; Frankstahl; Massmutual; Clean Energy...,Energy,Energy Storage,Recycling
89,40145579,163286439,Camp Six Labs,campsix.com,"Make big hardware prototypes in hours, not weeks.",Hospitality; Logistics; Delivery; Drones; Infr...,Lemnos; Holt Ventures,Built Environment; Energy,Renewable Intermittent Generation; Software / ...,Analytics


In [3]:
organizations

,Organization Id,Name,Website
0,95105353,Blackhorn Ventures,blackhornvc.com
1,10980507,URBAN-X,urban-x.com
2,282610437,C3,joinc3.co
3,1680908,160 Varick St.,poly.edu
4,1685981,Williams College,williams.edu
...,...,...,...
9509,286780017,TerraNexum,terranexum.com
9510,286810525,Backbone Ventures,backbone.vc
9511,286812958,Proprio,propriorobotics.xyz
9512,286944079,AI Robo Sense,airobosense.com


In [6]:
def enrich_investors(deals_df, organizations_df):
    field_list = list(set(list(deals_df["Investors"].unique())))
    option_list = []
    for entry in field_list:
        option_list += [x.strip() for x in str(entry).split("; ")]
    investor_list = list(set(option_list))
    ''' 
    at this point, there are several duplicates in the list with slightly different names
    generally this is not a big deal, since we're going to be merging this list into the
    organization list anyway, and we shouldn't get duplicates there unless our organization
    list has duplicates (which we shouldn't, and if we do, affinity will suggest fixing it)
    '''
    investor_df = pd.DataFrame(investor_list, columns=['Name'])

    # pull out the Sector Sub-Sector and Tech Trend(s) from each portco, and map to investors
    for investor in investor_df['Name']:
        sector_dict = {}
        subsector_dict = {}
        tech_dict = {}
        for _, row in deals_df.iterrows():
            if investor in row['Investors']:
                sector_dict[str(row['Sector'])] = sector_dict.get(str(row['Sector']), 0) + 1
                subsector_dict[str(row['Sub-Sector'])] = subsector_dict.get(str(row['Sub-Sector']), 0) + 1
                tech_dict[str(row['Tech Trend(s)'])] = tech_dict.get(str(row['Tech Trend(s)']), 0) + 1
        sector_dict.pop('nan',None)
        subsector_dict.pop('nan',None)
        tech_dict.pop('nan',None)
        investor_df.loc[investor_df['Name'] == investor, 'Sector'] = '; '.join(sorted(sector_dict, key=sector_dict.get, reverse=True))
        investor_df.loc[investor_df['Name'] == investor, 'Sub-Sector'] = '; '.join(sorted(subsector_dict, key=subsector_dict.get, reverse=True))
        investor_df.loc[investor_df['Name'] == investor, 'Tech Trend(s)'] = '; '.join(sorted(tech_dict, key=tech_dict.get, reverse=True))
        '''NOTE: this creates duplicates of each field in the investor list, but this doesn't matter because Affinity squashes duplicates'''

    # merge organization list websites to investor list
    # first create a lowercase name for both the organization and investor list
    organizations_df['lower'] = organizations_df['Name'].str.lower()
    investor_df['lower'] = investor_df['Name'].str.lower()
    # merge the two lists on the lowercase name
    enriched_investors_df = pd.merge(organizations_df[['Name', 'Website', 'lower']], investor_df[['lower', 'Sector','Sub-Sector','Tech Trend(s)']], on='lower')
    # drop the lowercase column
    enriched_investors_df = enriched_investors_df.drop(columns='lower')
    #enriched_investors_df.to_csv('explore/enriched_investors.csv')
    return enriched_investors_df

In [8]:
enriched_investors = enrich_investors(deals, organizations)
enriched_investors.to_csv('enriched_investors.csv')
enriched_investors

,Name,Website,Sector,Sub-Sector,Tech Trend(s)
0,Blackhorn Ventures,blackhornvc.com,Built Environment; Cross-Sector; Built Environ...,Transmission & Distribution Maintenance; Data ...,Internet of Things; Analytics; Analytics; Voic...
1,Muus Asset Management,muus.com,Energy,DER Asset Integration with Wholesale Markets,Analytics
2,Riverstone Holdings,riverstonellc.com,Energy,DER Asset Integration with Wholesale Markets,Analytics
3,Blue Bear Capital,bluebearcap.com,Energy; Cross-Sector; Built Environment,Data Capture and Data Fusion for High-Level In...,Analytics; Voice AI
4,Elemental Excelerator,elementalexcelerator.com,Energy; Transportation + Logistics,EV Ecosystem,Labor Training
...,...,...,...,...,...
67,Ubiquity Ventures,ubiquity.vc,Transportation + Logistics,,
68,CEAS Investments,ceasinvestments.com,,,
69,Climate Capital,climatecapital.co,Energy,DER Asset Integration with Wholesale Markets,Analytics
70,VoLo Earth Ventures,voloearth.com,Energy,Energy Storage,Recycling
